**Collaborative filtering**

**Distributed Stochastic Gradient Descent**
In this method,we want to parellize the task of recieving stream data and giving it to many child processes for parelllel processing

This can be achieved by dividing the rating matrix R into blocks for rach child processes


**Importing Libraries**

In [0]:
#Importing Libraries

from sklearn import metrics
import pandas as pd
import numpy as np
from pprint import pprint
from sklearn.datasets import load_boston
from numpy.linalg import inv, pinv, LinAlgError

**The dataset consists of user ratings for various item and the dataset is a sparse dataset**

The dataset can be downloaded from the below cell

In [30]:
!curl -O http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4808k  100 4808k    0     0  9559k      0 --:--:-- --:--:-- --:--:-- 9559k
Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


**Reading the dataset and considering only first 2000 rows and doing preprocessing**

In [0]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('/content/ml-100k/u.data', sep='\t', names=names)

# Split the dataframe into a train and test set
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df,test_size=0.3)


train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)


In [32]:
n_u = len(df["user_id"].unique())
n_m = len(df["item_id"].unique())
df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [0]:
n = 25
m = 25
R1 = np.zeros((n_u,n_m))
for line in train_data.itertuples():
    R1[line[1]-1, line[2]-1] = line[3]  
    
R = np.zeros((n,m))

T1 = np.zeros((n_u, n_m))
for line in test_data.itertuples():
    T1[line[1]-1, line[2]-1] = line[3]

T = np.zeros((n,m))

In [0]:
k = 10  # Number of latent factor pairs
lmbda = 0.5 # Regularisation strength
gamma= 0.01  # Learning rate
n_epochs = 40  # Number of loops through training data
U = 3 * np.random.rand(k, n) # Latent factors for users
V = 3 * np.random.rand(k, m) # Latent factors for items

# pprint(U)
# pprint(V)

In [0]:
# Global Variables for Updation
U_temp = np.zeros((k, n))
V_temp = np.zeros((k, m))

In [0]:
def thread_sgd(lock, R, U, V, p, q, d):
        
    lmbda = 0.1           # Penalty Parameter for regularization
    gamma = 0.001   # Step Size
        

    users,items = R.nonzero()      
    for epoch in range(n_epochs):
      for i, j in zip(users,items):
          error = R[i, j] - np.dot(U[:,i].T, V[:,j])  # Error for this observation
          U[:,i] += gamma * ( error * V[:,j] - (lmbda/m) * U[:,i]) # Update this user's features
          V[:,j] += gamma * ( error * U[:,i] - (lmbda/n) * V[:,j])  # Update this item's features
    #return U, V
    if p==0:
        start = 0
        end = d
    else:
        start = d*(p-1)
        end = d*p

    U_temp[:,start:end] = U
    V_temp[:,d*(q-1):d*q] = V





In [40]:
# Applying Distributed Stochastic Gradient Descent

from itertools import permutations
import threading

d = 5  # Number of processors
permutation_set = list(permutations(range(1,6)))
lock = threading.Lock() # Defining Lock For Threading

for strata in permutation_set:
        
    p=0
    q=strata[p]
    start = 0
    end = d
    t1 = threading.Thread(target=thread_sgd, args=(lock,R[start:end, d*(q-1):d*q], U[:, start:end], V[:, d*(q-1):d*q], p, q, d)) 
    
    p=1
    q=strata[p]
    start = d*(p-1)
    end = d*p
    t2 = threading.Thread(target=thread_sgd, args=(lock,R[start:end, d*(q-1):d*q], U[:, start:end], V[:, d*(q-1):d*q], p, q, d)) 
    
    p=2
    q=strata[p]
    start = d*(p-1)
    end = d*p
    t3 = threading.Thread(target=thread_sgd, args=(lock,R[start:end, d*(q-1):d*q], U[:, start:end], V[:, d*(q-1):d*q], p, q, d)) 
    
    p=3
    q=strata[p]
    start = d*(p-1)
    end = d*p
    t4 = threading.Thread(target=thread_sgd, args=(lock,R[start:end, d*(q-1):d*q], U[:, start:end], V[:, d*(q-1):d*q], p, q, d)) 
    
    p=4
    q=strata[p]
    start = d*(p-1)
    end = d*p
    t5 = threading.Thread(target=thread_sgd, args=(lock,R[start:end, d*(q-1):d*q], U[:, start:end], V[:, d*(q-1):d*q], p, q, d)) 

    # start threads 
    t1.start() 
    t2.start() 
    t3.start() 
    t4.start() 
    t5.start() 

    t1.join() 
    t2.join() 
    t3.join() 
    t4.join() 
    t5.join() 
            
    U = U_temp
    V = V_temp

utv_after = np.dot(U.T, V)
print("MAE : ", metrics.mean_absolute_error(T, utv_after))
print("MSE : ", metrics.mean_squared_error(T, utv_after))    

MAE :  19.198133697019816
MSE :  495.4817020973104
